In [1]:
import os
from planner import PseudoPlanner


In [2]:
base_dir = "../data/raw_plan_traces"
output_filepath = "../data/plain_traces/plan.txt"

def parse_plan_file(file, planner):
    with open(file, 'r') as f:
        lines = f.readlines()
    plan = []
    for line in lines:
        if line.startswith(";"):
            break
        action = line.strip().strip("()").split(" ")
        action_name = action[0]
        op = planner.domain.get_action(action_name)
        assert op is not None, f"Action {action_name} not found in domain"

        args = action[1:]
        arg_types = [p.type_name for p in op.parameters]
        arg_with_types = [arg + "?"+ t for arg, t in zip(args, arg_types)]
        new_action = f"({action_name} {' '.join(arg_with_types)})"
        plan.append(new_action)
    return plan


output_data = []
for domain in os.listdir(base_dir):
    print(domain)
    if not os.path.isdir(os.path.join(base_dir, domain)):
        continue
    planner = PseudoPlanner(os.path.join(base_dir, domain, "domain.pddl")) 
    for dir in os.listdir(os.path.join(base_dir, domain)):
        
        if dir == "training":
            for file in os.listdir(os.path.join(base_dir, domain, dir, "easy")):
                if file.endswith(".plan"):
                    plan = parse_plan_file(os.path.join(base_dir, domain, dir,"easy", file), planner)
                    data = f"{domain}&&{"easy"}&&{file}&&{len(plan)}&&{','.join(plan)}\n"
                    output_data.append(data)
        elif dir == "testing":
            for difficulty in os.listdir(os.path.join(base_dir, domain, dir)):
                for file in os.listdir(os.path.join(base_dir, domain, dir, difficulty)):
                    if file.endswith(".plan"):
                        plan = parse_plan_file(os.path.join(base_dir, domain, dir, difficulty, file), planner)
                        data = f"{domain}&&{difficulty}&&{file}&&{len(plan)}&&{','.join(plan)}\n"
                        output_data.append(data)


try:
    with open(output_filepath, 'w', buffering=1) as file:  # Line buffered
        for line in output_data:
            file.write(line)
except Exception as e:
    print(f"Error writing to file {file_path}: {e}")

childsnack
n-puzzle
transport
ferry
rovers
spanner
miconic
floortile
blocksworld
satellite
sokoban
